In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

hbar = 1
m = 1
L = 1
N = 300
dx = L / (N + 1)
x = np.linspace(dx, N * dx, N)

DEBUG = False

# Some does not have scipy.linalg.eigh_tridiagonal, so this takes care of that
try:
    from scipy.linalg import eigh_tridiagonal
    get_eigh = lambda H: eigh_tridiagonal(np.diag(H), np.diag(H, k=1))
except ImportError:
    get_eigh = np.linalg.eigh


def V(x):
    return 100*np.heaviside(x-0.4, 0.5) * np.heaviside(0.6-x, 0.5)

# Initialize the tridiagonal NxN matrix H
H = np.zeros((N, N))
for i in range(0, N):
    H[i][i] = hbar**2 / (m * dx**2) + V(x[i])
for i in range(0, N - 1):
    H[i][i+1] = -hbar**2 / (2 * m * dx**2)
    H[i+1][i] = -hbar**2 / (2 * m * dx**2)

eigvals, eigvecs = get_eigh(H)

if DEBUG:
    print(np.linalg.norm(eigvecs[:,10]))

# Plot the first 5 energy levels
for j in range(0, 5):
    eigval = eigvals[j]
    plt.plot((0, L/2), (eigval, eigval))
    eigval = hbar**2 * ((j+1) * np.pi / L)**2 / 2 / m # Theoretical values for V(x) = 0.
    plt.plot((L/2, L), (eigval, eigval))
plt.show()

j = 8
y1 = eigvecs[:,j-1]
y2 = np.sqrt(2/L) * np.sin(j*np.pi/L*x)
plt.plot(x, y1, "k-")
plt.plot(x, y2 * dx**(1/2), "r--")
plt.show()


waves = eigvecs.T

plt.plot(x, V(x))
for j in range(0, 5):
    eigval = eigvals[j]

# Plot potential, energy levels and eigenfunctions in one plot
# TODO: as of now, the scaling is ad-hoc
NUM_VALS_PLOT = 5
for wave,engy in np.array(list(zip(waves, eigvals)))[:NUM_VALS_PLOT]:
    plt.plot((0, L), 2*[engy])
    plt.plot(x, 120*wave + engy)

print(y1[0] / y2[0], dx**(1/2), 1/dx)

if DEBUG:
    for i in range(0, 5):
        for j in range(0, 5):
            print("row %d (dot) row %d: %.2f" % (i, j, np.dot(eigvecs[i,:], eigvecs[j,:])))
            print("col %d (dot) col %d: %.2f" % (i, j, np.dot(eigvecs[:,i], eigvecs[:,j])))
        

